# BES INDICATORS IN ITALY

The [__Bes project__](https://www4.istat.it/en/well-being-and-sustainability/well-being-measures/bes-report) 
was launched in 2010 to measure Equitable and Sustainable Well-being,
and with the aim of evaluating the progress of society not only from an economic,
but also from a social and environmental point of view. 


To this aim, the traditional economic indicators, GDP first of all, have been integrated with measures of the quality of people’s life and of the environment.


Since 2016, well-being indicators and welfare analyzers have been presented with indicators for monitoring the objectives of the 2030 Agenda
for Sustainable Development,
the so-called [Sustainable Development Goals (SDGs)](https://sdgs.un.org/goals) of the United Nations.
They were chosen by the global community through a political agreement between the different actors, to represent their values, priorities and objectives.
The United Nations Statistical Commission (UNSC) has set up a shared set of statistical information to monitor the progress of individual countries towards the SDGs,
including over two hundred indicators.
The two sets of indicators are only partially overlapping, but certainly complementary.


Bes' indicators cover [12 domains relevant for the measuramente of the well-being](https://www.istat.it/it/files/2018/04/12-domains-scientific-commission.pdf) 
and they are the following: 

1)  Health 
2)  Education & Training
3)  Work & Life Balance
4)  Economic well-being
5)  Social Relationships
6)  Politics & Istitutions
7)  Security 
8)  Subjective well-being
9)  Landscape & Cultural heritage
10) Environment
11) Innovation, Research & Creativity
12) Quality of services

These twleve indicators are themselves subdivided into smaller subindicators and each of them refer to a specific measure.


## Data collection

The data dowloaded from the national institute of statistics is organized in a huge excel file. This file contains the data for all the indicators. 
<br>
In order to analyze and represent the data there is the need to split this file in smaller files: One for each subindicator.

In [ ]:
# import the dependencies
import pandas as pd
import os

In [ ]:
# Import the data
dati_bes = pd.read_excel('D:\Geospatial_Project\Dati Bes\Bes\Indicatori_per_provincia_sesso.xlsx')

In [ ]:
# Create an empty dataframe
header = dati_bes.columns
tabella_new = pd.DataFrame(columns=header)

To split the data in an automatic manner there is the need to create an algorithm that detects whether there is a change in the unit of measure or in the indicator.
<br>
A smart way to do so is by looking at the territories. Indeed, there is a change in the subindicators everytime the data refer to ´Italy´ which is the average that for that specific measure. 

In [ ]:
# The pattern that change 
for j in range(len(dati_bes)):
    mini_ds = dati_bes.iloc[j]
    tabella_new = tabella_new.append(mini_ds)
    if mini_ds['TERRITORIO'] == 'Italia':
        Dominio = dati_bes.iloc[j]['DOMINIO']
        Indicatore = dati_bes.iloc[j]['INDICATORE']
        Sesso = dati_bes.iloc[j]['SESSO']
        Unita = dati_bes.iloc[j]['UNITA_MISURA']
        tabella_new.to_excel('D:/Geospatial_Project/Nuovi_Dati2/{Dominio}-{Indicatore}-{Sesso}-{Unita}.xlsx'.format(Dominio=Dominio, Indicatore=Indicatore, Sesso = Sesso, Unita = Unita))
        tabella_new = pd.DataFrame(columns=header)

In [ ]:
# check whether the data has been properly stored
files_ = os.listdir(path='Nuovi_Dati2')
files_

In [ ]:
#check whether all the datasets have the same lengths. If not there is the need to 
#understand why
to_chekc = []
for file in files_:
    if file.endswith('.xlsx'):
        excel_ = pd.read_excel('Nuovi_Dati2/'+file)
        if len(excel_) != 135:
            print(file + str(len(excel_)))
            to_chekc.append(file)

As we can see there are some datasets whose lengths is sligthly different compared to others. 
<br>
Some hypothesis can be made in order to justify these differences:
* maybe the province does not send the required data and so it has not been considered;
* the province did not exist when the data collection started (the reference year is 2004) 
<br>

To understand the real cause there is the need to see which territories are missing.

In [ ]:
df_prova = pd.read_excel('Nuovi_Dati2/Benessere economico-Retribuzione media annua dei lavoratori dipendenti-Totale-euro.xlsx')
s2 = df_prova.TERRITORIO.unique().tolist()
path = 'Nuovi_Dati2/'
Territori_Mancanti = {}
for f in to_chekc:
    df_check = pd.read_excel(path + f)
    s1 =df_check.TERRITORIO.unique().tolist()
    mis_val = set(s2) - set(s1) 
    Territori_Mancanti[f] = mis_val

In [ ]:
Territori_Mancanti

From the above cell, it is possible to notice that all the missing territories refer to the region of Sardinia. Hence it is possible to believe that the 'missing territories' refer to territories that have changed their territorial administration or something like that.   

##  Data Analysis

Since the data has been stored in smaller files, one for each subindicators, it is possible to start the analysis and the representation of the data. 
<br>
Firstly, there is the need to decide how do we want to organize the analysis: Indeed, we have data of related to provinces, regions and macroareas (north, centre, south) all together. Thus, there is the need to split the data in three subsets, one for each level of analysis:
*  Macrolevel : A comparison between North, Center, and South of Italy:
    * North = Piemonte, Valle d'Aosta, Lombardia, Liguria, Trentino-Alto Adige, Veneto, Friuli-Venezia-Giulia, Emilia-Romagna
    * Center: Toscana, Umbria, Marche, Lazio
    * South: Abruzzo, Molise, Campania, Puglia, Basilicata, Calabria, Sicilia, Sardegna
* Regional level: A comparison among the twenty italian regions
* Province level : A comperison among the one hundred and seven italian provinces. 
<br>


Since we will end up with many differnt datasets I choose to create functions that clean and manipulate the datasets in an automatic manner and according to the diffent needs. 
<br>
To clarify we will start having ordinary pandas dataframes and the output will be a geopandas dataframe that will allow us to create geospatial representation and to perform some kind of data analysis.  

In [ ]:
# required dependencies
import os 
import esda
import math
import folium
import leafmap
import numpy as np 
import pandas as pd
import seaborn as sbn
import libpysal as lps
import geopandas as gpd

### Functions 

In [ ]:
def read_dati_bes(path):
    
    '''
    Through this function the excel file:
    * is read and imported;
    * is reduced by dropping unuseful columns     
    '''

    df = pd.read_excel(path)
    df.drop(columns = ['Unnamed: 0', 'DOMINIO', 'CODICE', 'SESSO', 'FONTE'], inplace = True)

    return df 


def provinces_BES(df, Anomalous_Regions, Sud_Sardinia, Macro_Areas, Italy, Regions):

    '''
    Due to the difference in annotation between the BES_Indicator data and the one of ISTAT,
    there is the need to change some of the names of the provinces.
    Firstly, we should brake the dataset of BES Indicators in order to separate regions and macro areas,
    NB: We have to check that there are NO differences between regions'names
    Secondly, we have to omologate the names of the two dataset.
    NB: There are substantial differences mainly involving Sardinia, where ISTAT's Sud-Sardegnia contains BES'S Sud-Sardegna, Medio-Campidano, Carbonia-Iglesias, Ogliastra and Olbia-Tempio.
    '''

    #all_territories = df.TERRITORIO.unique().tolist()
    #return set(all_territories) - set(Regions) - set(Anomalous_Regions) - set(Sud_Sardinia) - set(Macro_Areas) - set(Italy)
    all_territories = df.TERRITORIO.unique().tolist()
    territories =  set(all_territories) - set(Regions) - set(Anomalous_Regions) - set(Sud_Sardinia) - set(Macro_Areas) - set(Italy)
    territories = list(territories)
    territories.sort()
    return territories

def regions_BES(Regions):
    '''
    Due to the difference in annotation between the BES_Indicator data and the ones of ISTAT,
    there is the need to change some of the names of the regions.
    '''

    new_regions = []
    for reg in Regions:
        if reg =="Valle d'Aosta":
            new_regions.append("Valle d'Aosta/Vallée d'Aoste")
        elif reg == 'Trentino-Alto Adige':
            new_regions.append("Trentino-Alto Adige/Südtirol")
        elif reg == 'Friuli Venezia Giulia':
            new_regions.append('Friuli-Venezia Giulia')
        else:
            new_regions.append(reg)
    
    return new_regions


def mod_col_geo(geodf):
    
    '''
    This function drops useless columns to the geodataframe and rename one of its column
    '''
    
    geodf.drop(columns=['COD_RIP','COD_REG'], inplace = True)
    geodf.rename(columns={'DEN_REG':'Reg'}, inplace = True) 
    
    return geodf

def get_regions_names(geodf):  

    '''
    This function returns a list of all the regions names according to the ISTAT standard
    NB there are some differences among the names used by ISTAT and the ones used for the BES indicators 
    '''

    return  geodf.Reg.unique().tolist() 

def sort_reset_index_geo(geo_df):

    '''
    The geodaframe is sorted and its indeces are reset
    (This will be useful when the main dataframe will be converted into a geopandas dataframe)
    '''
    geo_df.sort_values(by = ['Prov'], inplace = True) 
    geo_df.reset_index(inplace = True)

    return geo_df

def order_df(df, BES_Territories):

    '''
    The daframe is:
    * reduced in order to have coherence with the geodatafrmae which considers less provinces than the one of bes (actually, the geodatraframe respects the actual division of Italy into provinces)
    * sorted
    * its indeces are reset
    (This will be useful when the main dataframe will be converted into a geopandas dataframe)
    '''

    prov_df =  df[df['TERRITORIO'].isin(BES_Territories)]

    if len(prov_df) > 3:
        prov_df.sort_values(by=['TERRITORIO'],inplace = True)
        
    
    prov_df.set_index('TERRITORIO', inplace = True)

    return prov_df


def clean_prov_geo(geo_prov, provinces):

    '''
    This functions cleans the geodataframe relative to provinces. 
    What this function does is:
    * dropping useless columns
    * sorting values according to territories' names
    * setting the territories names as indexes
    '''
      
    geo_prov.drop(columns = ['COD_RIP','COD_REG',	'COD_PROV',	'COD_CM',	'COD_UTS',	'DEN_PROV',	'DEN_CM', 'TIPO_UTS'])
    geo_prov.sort_values(by = 'DEN_UTS', inplace = True)
    geo_prov.DEN_UTS = provinces
    geo_prov.set_index('DEN_UTS', inplace = True)
  
    return geo_prov

def order_df_regions(geodf, BES_Territories):
    '''
    This function modifies the geodaframe in order to have the same nomenclature for both the df and the geodf.
    This step  is needed in order to merge the two at the next step.
    '''
    
    geodf['Reg'] = BES_Territories #CHANGE
    geodf.sort_values(by= ['Reg'], inplace = True)
    geodf.set_index('Reg', inplace = True )
    
    return geodf

def aggregate_macros(geodf):
    
    '''
    The data regarding BES indicators refer to just three macroareas insted of 6 as in the ISTAT dataset. 

    Hence, there is the need to marge the areas in order have 3 main areas

    1.   North : Piemonte, Valle d'Aosta/Vallée d'Aoste, Lombardia, Liguria, Trentino-Alto Adige/Südtirol, Veneto, Friuli-Venezia Giulia, Emilia-Romagna
    2.   Centre: Toscana, Umbria, Marche, Lazio
    3.   South(& Islands): Abruzzo, Molise, Campania, Puglia, Basilicata, Calabria, Sicilia, Sardegna  
    '''

    Territorio = ['Nord', 'Nord', 'Centro', 'Mezzogiorno', 'Mezzogiorno']
    geodf['TERRITORIO'] = Territorio
    geodf = geodf.to_crs(epsg=4326).dissolve(by='TERRITORIO')
    geodf.drop(columns = ['DEN_RIP'])
    geodf.sort_values(by = ['COD_RIP'], inplace = True)

    return geodf

def from_df_to_gdf(df, geo_df):

    '''
    With this function we convert the dataframe containing the statistics we are interested in into a geodaframe. 
    This allows us to use all the functionalities of a geopandas dataframe such us doing plots.
    '''

    df['Shape_Leng'] = geo_df['Shape_Leng']
    df['Shape_Area'] = geo_df['Shape_Area']
    df['geometry'] = geo_df['geometry']
    df = gpd.GeoDataFrame(df)

    return df

def look_for_anomalies2(df,var):

    i = 0
    in_set = set()

    for el in df[var].values:
        if type(el) == str or math.isnan(el): 
            
            in_set.add(i)
    
        i += 1 

    df.reset_index(inplace=True)
    df.drop(list(in_set), inplace = True)   
    
    
    return df

## Create the geodataframes

As said above, we have data regarding the different subindicators. However, in order to perform geospatial analyses and create geospatial representations there is the need to have the data about the ´geometries´ of the territories we are dealing with. 
<br>
Thus, the first thing to do is importing the geodataframes provided by the national institute of statistics.

In [ ]:
# Path where are stored the shape files
Reg_Path = 'D:/Geospatial_Project/Dati/Limiti/Lim/Reg01012021/Reg01012021_WGS84.shp'
Prov_Path = 'D:/Geospatial_Project/Dati/Limiti/Lim/ProvCM01012021/ProvCM01012021_WGS84.shp'
Macro_Path = 'D:/Geospatial_Project/Dati/Limiti/Lim/RipGeo01012021/RipGeo01012021_WGS84.shp'
# Read the shape files and create the geodataframes
# One for each territorial division (Macroareas, Regions and provinces)
Reg_df = gpd.read_file(Reg_Path)
Prov_df = gpd.read_file(Prov_Path)
Macro_df = gpd.read_file(Macro_Path)

Now, some lists containing the names of problematic territories are created. 
These lists will be used to create the geodataframes. Indeed, having the right and common nomenclature is needed in order to merge the pandas dataframe with the geopandas dataframe. 

In [ ]:
# Anomalous Regions names
Anomalous_Regions = ['Trentino-Alto Adige/Südtirol','Friuli-Venezia Giulia',"Valle d'Aosta/Vallée d'Aoste"]
#Sud_Sardinia_Agglomeration
Sud_Sardinia = ['Ogliastra','Olbia-Tempio','Medio Campidano','Carbonia-Iglesias']
# Macro_Areas 
Macro_Areas = ['Centro', 'Mezzogiorno', 'Nord']
# Italy 
Italy = ['Italia']
# Different_Names
Different_Names_BES = ['Bolzano/Bozen', 'Forlì-Cesena', 'Massa-Carrara', 'Reggio Calabria']
Deffirent_Names_ISTAT = ['Bolzano', "Forli'-Cesena", 'Massa Carrara', 'Reggio di Calabria']

With the following chunk of code we create all the python objects needed to obtain the geodaframes.

In [ ]:
# Read the BES_Statistics Dataframe
path_ = 'D:/Geospatial_Project/Nuovi_Dati2/'
List_Statistics = os.listdir(path = path_)
# Read a random file just to compute the provinces afterward
df_ = read_dati_bes(path_ + '/' + List_Statistics[0])
# Get the Istat regions 
Regions = Reg_df.DEN_REG.to_list()
# Get  BES Regions
Bes_Regions = regions_BES(Regions)
# Extract the set of all Bes provinces
provinces = provinces_BES(df_, Anomalous_Regions, Sud_Sardinia, Macro_Areas, Italy, Regions)

### Provinces

Let's start by creating the geodataframe related to provinces. 
<br>
To do so we choose a random datasets from the set of all the dataframes. 

In [ ]:
# Import any datasets relative to the one of the subindicators
stat = List_Statistics[0]
df = read_dati_bes(path_+ '/' + stat)

In [ ]:
## PREPARE PROV GEO DF ## 
Prov_df = clean_prov_geo(Prov_df, provinces)
# Create the provinces df
df_prov = order_df(df, provinces)
# Obtain the full geodataframe of provinces 
df_prov = from_df_to_gdf(df_prov, Prov_df)

In [ ]:
# visualize the datasets
df_prov.head()

### Regions

As we have done for provinces we will create our geodataframe relative to the regions. 

In [ ]:
#This chunk of code does not need to be executed if the one above has been executed

# Import any datasets relative to the one of the subindicators
stat = List_Statistics[0]
df = read_dati_bes(path_+ '/' + stat)

In [ ]:
## PREPARE REG GEO DF ##
Reg_df = mod_col_geo(Reg_df)
Reg_df = order_df_regions(Reg_df, Bes_Regions)
# Creating Regions DataFrame
df_reg = order_df(df, Bes_Regions)
# Create the geodaframe used in the representation
df_reg = from_df_to_gdf(df_reg, Reg_df)

### Macroareas

Lastly, we will create the macroareas geodataframe.

In [ ]:
## PREPARE MACRO GEO DF ##
Macro_df = aggregate_macros(Macro_df)
# 'MACRO-AREA' #
# Creating the MacroArea Dataframe
df_macro = order_df(df, Macro_Areas)    
# Create the geodaframe used in the representation
df_macro = from_df_to_gdf(df_macro, Macro_df)

## Representation

Now that we have created all the geodataframes, it is possible to create some representation. 
Two kind of geospatial plots will be created:
* A static choropletmap via Matplotlib that will show the data (if specified) grouped in quantiles
* An interactive choropletmap created with folium

In [ ]:
import plotly.express as px
from matplotlib import colors
import matplotlib.pyplot as plt
from folium.plugins import HeatMap
from folium.plugins import MarkerCluster

As did before, to avoid long and repetitive chunks of code will be defined some functions at the top that will be then called to create the needed plots. 

In [ ]:
def get_title_(file_name):
    
    '''
    This function provides the title to plots 
    '''

    title = file_name.strip('.xlsx')
    title = title.split('-')
    title = title[0].upper() + ' ' + title[1] + ' ' + title[2]

    return title 

def get_labels_(df):
    
    '''
    This functions returns the indicators that will show up in the folium map while using the pointer
    over the territories involved in the analysis
    '''
    
    return df.iloc[0]['INDICATORE'] + '\n' + '(' + df.iloc[0]['UNITA_MISURA'] + ')'

def static_choroplet(df, variable, title_, K = 5, Scheme = 'equal_interval'):

    '''
    With this function is it possible to create a static Choroplet map 
    Additional feature that can be added are ad-hoc colormaps, and a scheme for the division of the classes.
    '''
    
    miss_df = df[df[variable].isna()]
    df.dropna(subset = [variable], inplace = True)
    if len(df) > 3:
        df = look_for_anomalies2(df, variable)
    ax = df.plot(column=variable,
                                cmap="OrRd", edgecolor = "darkgrey", 
                                linewidth = 0.9,legend=True,figsize=(14,14),
                                scheme=Scheme,
                                k=K)
    if len(miss_df) > 0 : 
        miss_df.plot(color = 'gray', ax = ax)
    ax.set_axis_off()
    ax.set_title(title_)
    #plt.show()
    fig = ax.figure
    return fig   # modifica 24/04 


def dynamic_choroplet(df, title_, measure):
    
    '''
    # NOT USED
    With this function it is possible to create an interactive choropletmap
    '''

    fig = px.choropleth(df,
                   geojson=df.geometry,
                   locations=df.index, # maybe this should be changed for provinces 
                   color=measure,
                    title = title_)
    fig.update_geos(fitbounds="locations", visible=False)
    #fig.show()
    return fig



def folium_interactive_map(df, var, file_name, indicator):
    
    '''
    This function will create an interactive folium choropletmap. Moreover, this maps provides also a feature
    that allows to see the specific measure when the reader uses the pointer over the territories involved
    in the analysis
    '''
    

    # Create the folium map 
    m10=folium.Map(location=[41.9027835,12.4963655],tiles='openstreetmap',zoom_start=5)

    df = look_for_anomalies2(df, var)
    df.to_crs(4326, inplace = True)

    
    folium.Choropleth(
    geo_data = df.to_json(),
    data = df,
    columns=['TERRITORIO', var],
    key_on='feature.properties.TERRITORIO',
    #key_on = 'feature.properties.id',
    fill_color='Oranges', 
    fill_opacity=0.6, 
    line_opacity=1,
    nan_fill_color='black',
    legend_name= get_title_(file_name),
    smooth_factor=0).add_to(m10)


    
    #add the feature
    folium.features.GeoJson(df,
                        name='Labels',
                        style_function=lambda x: {'color':'transparent','fillColor':'transparent','weight':0},
                        tooltip=folium.features.GeoJsonTooltip(fields=[var],
                                                                aliases = [indicator], #Substitute with the indicator of the df
                                                                labels=True,
                                                                sticky=False
                                                                            )
                       ).add_to(m10)
    

    return m10

def look_for_anomalies2(df,var):

    '''
    This functions is foudnamental to properly create the geospatial repersentations. 
    Indeed, in the original file dowloaded from the national istitute of statistics, missing values are treated 
    in different ways: some missing values are reported as a string like that '...', 
    others are in the format np.nan and lastly some are in the formar math.nan.
    This functions simply drops the missing values for the specific year of analysis. 
    '''

    i = 0
    in_set = set()

    for el in df[var].values:
        if type(el) == str or math.isnan(el): 
            
            in_set.add(i)
    
        i += 1 

    df.reset_index(inplace=True)
    df.drop(list(in_set), inplace = True)   
    
    
    return df



Not it is possible to create a plot relative to a specific subindicators in a given year. 

In [ ]:
# Creating the title
titolo = get_title_(stat)
# Get_labels 
labels = get_labels_(df)

In [ ]:
# select a year in which we are interested to know about
y_ = 'V_2016'
# Create the static choropletmat for each of the geodaframe
matplotlib_fig = static_choroplet(df_macro, variable = y_, title_ = titolo,)
matplotlib_fig2 = static_choroplet(df_reg, variable = y_, title_ = titolo)
matplotlib_fig3 = static_choroplet(df_prov, variable = y_, title_ = titolo)

In [ ]:
# Create the folium interactive choroplet maps 
folium_interactive_map(df_macro, y_, stat, labels)

In [ ]:
folium_interactive_map(df_reg, y_, stat, labels)

In [ ]:
folium_interactive_map(df_prov, y_, stat, labels)

Calling one of the above functions will display the folium map

Since the above representation show the data for a specific measure in a fixed period of time, it can be useful to create an additional plot, a line chart, that helps to understand the trend of that specific measure. 
<BR>
To do so we will use plotly, a python library that creates interactive plots. 
Indeed, this kind of chart allows to visualize the data of a specific territory by clicking on the territories in the right slider menù. Depending on the different level of analysis it is possible to decide which observation has to be hided

In [ ]:
def line_chart_plotly(Bes_df, to_hide_1, to_hide_2, to_hide_3, titolo = ''):
    
    '''
    This functions create a dataframe related to a specific measure where each observation refers to a 
    differnet year. 
    Then uses this daatframe to create a linechart. 
    
    '''

    df_plotly = pd.DataFrame(columns = ['TERRITORIO', 'MISURA', 'ANNO'])
    #if len(df_plotly) > 1:
    territorio = []
    misura = []
    _anno_ = []
    for i in range(len(Bes_df)):
        for j in range(4,20):
                
            if j < 10:
                anno = 'V_200' + str(j)
            else:
                anno = 'V_20' + str(j)
                    
            anno_ = int(anno[2:])
            territorio.append(Bes_df.iloc[i]['TERRITORIO'])
            misura.append(Bes_df.iloc[i][anno])
            _anno_.append(anno_)

        
        
    df_plotly['TERRITORIO'] = territorio
    df_plotly['ANNO'] = _anno_
    df_plotly['MISURA'] = misura


    to_hide_1.extend(to_hide_2)
    to_hide_1.extend(to_hide_3)
    fig = px.line(df_plotly, x = 'ANNO', y = 'MISURA', color = 'TERRITORIO', title = titolo)
    fig.for_each_trace(lambda trace: trace.update(visible="legendonly") 
            if trace.name in to_hide_1 else ())
        
        
    return fig

In [ ]:
#Macros
line_chart_plotly(df, to_hide_1 = provinces, to_hide_2 = Bes_Regions, to_hide_3 = Sud_Sardinia, titolo = titolo)

In [ ]:
#Regions
line_chart_plotly(df, to_hide_1= provinces, to_hide_2 = Macro_Areas, to_hide_3 = Sud_Sardinia, titolo = titolo)

In [ ]:
#Provinces
line_chart_plotly(df, to_hide_1 = Bes_Regions, to_hide_2 = Macro_Areas, to_hide_3 = Sud_Sardinia, titolo = titolo)

## Additional Plots

In addition to the above choroplets maps, it is possible to create some additional representation that can support the above maps and maybe help to better understand/interpret the data.
For this purpose, data from Open street Map will be used to retrieve geodata. For instance, it is possible to collect the data about all the factories in Italy in order to understand if there is some correlation between the presence of many facotries and some indexes related to the enviroment (in the context of pollution) or to the economic wellbeing (many factories are connected to more employment?!). 
<br>
The starting point is dowloading the pbf files of the different regions and or provinces from [Wikimedia Italia](https://osmit-estratti.wmcloud.org/)
<br>
Secondly there is the data extraction via the osm library called pyrosm. 
<br>
Here it is important to specify two things if someone wants to execut the chunks of code in this seciton of the notebook:
* Since some Regions' PBF files were too big, it was infeasible for my local machine to run the code, hence I needed to download the PBFs files of all the provices belonging to these (big) regions. 
* To run this code it is necessary to have a virtual environment where pyrosm is installed. To do so I created a new envrinoment where I have installed as first package pyrosm. Geopandas was automatcally installed since it is one of the dependencies of pyrosm.  


<span style = "color:red"> NB: To execute the code in the above cell and the following. Due to problem with conflicting packages I created a venv with geopandas folium etc and onother one where I have just installed pyrosm. By installing pyrosm you also install Geopandas in a compatible version </span>.

In [ ]:
import os 
#import pyrosm
import pandas as pd 
import  geopandas as gpd
from folium.plugins import HeatMap
from folium.plugins import MarkerCluster

With the above chunk of code it is possible to extract points related to many different locations based on a filter that specifies in which kind of places we are interested in and save them into shp or csv files. 
The tags system is defined by OSM. 

In [ ]:
# Change the working directory to the one where are saved all the PBFs files dowloaded from Wikimedia Italia
os.chdir('D:/Geospatial_Project/Big_Cities')
# Create a list with all the pbfs files 
regions_pbf = os.listdir()
custom_filter = {'amenity' : ['srts_centre', 'brothel', 'casino', 'cinema',
                            'community_centre', 'conference_centre', 'events_venue', 'fountain',
                            'gambling', 'love_hotel', 'nightclub', 'planetarium', 'public_bookcase',
                            'social_centre', 'stripclub', 'studio', 'swingerclub', 'theatre']}
# Define the filter


# Iterate along all the regions (or provinces)
for region in regions_pbf:
    
    name = region.split('_')
    name = name[1]
    
    osm = pyrosm.OSM(region)
    data = osm.get_pois(custom_filter = custom_filter)
    
    try:
        data.to_file('D:/Geospatial_Project/Freetime/' + name + '/' + name + '.shp')
    except:
        data = pd.DataFrame(data)
        data.to_csv('D:/Geospatial_Project/Freetime/' + name + '.csv')

In [ ]:
def read_csv_shp(path):

    '''
    This function read the csv and shp files that contain the geodata retrived from osm.
    Then it merges all this files into a big datasets where are stored the latitude, longitude 
    and the name of the point of interest.    
    '''
    
    _full_df_ = gpd.GeoDataFrame(crs = 4326, columns = ['name', 'lat', 'lon'])
    lats = []
    lons = []
    names = []
    list_files = os.listdir(path)
    list_files_ = []


    for file_ in list_files:
        if file_.endswith('.csv'):
            list_files_.append(file_)
        elif file_.endswith('.shp'):
            list_files_.append(file_)


    for _file_ in list_files_:
        
        if _file_.endswith('.csv'):
            _df_ = pd.read_csv(path + '/' + _file_)
            if len(_df_) > 0:
                _df_['geometry'] = gpd.GeoSeries.from_wkt(_df_['geometry'])
                _geodf_ = gpd.GeoDataFrame(_df_, crs = 4326)
        
        elif _file_.endswith('.shp'):
            _geodf_ = gpd.read_file(path + '/' + _file_)
        lats.extend(_geodf_.geometry.centroid.to_crs(4326).y)
        lons.extend(_geodf_.geometry.centroid.to_crs(4326).x)

        if 'name' in _geodf_.columns:
            names.extend(_geodf_.name)
        else: 
            names.extend(['Missing'] * len(_geodf_.geometry.centroid.to_crs(4326).x))
    

    new_names = []
    for n in names: 
        if type(n) != str:
            new_names.append('Missing')
        else:
            new_names.append(n)
    

    _full_df_['lat'] = lats
    _full_df_['lon']= lons
    _full_df_['name'] = new_names

    return _full_df_

def getMarker(lat,lon, message,inconstyle):

    '''
    This function simply add a marker on a folium map based on the point latitude and logitude. 
    In addition, there is also a pop up message that shows up when the user interact with the map. 
    Lastly, it is even possible to specify an icon that defines the point on a map. 
    '''

    marker = folium.Marker(location=[lat,lon],
                         popup=message,
                         icon=inconstyle)
    return marker


def marker_plot(df, logo : str):
    '''
    This functions create a map. 
    Then it adds to this maps all the points present in the given datasets. 
    By iterating over the rwos of the provided dataset applies the function defined above 
    (getMarker) to add the marker. 
    '''

    m5=folium.Map(location=[41.9027835,12.4963655],tiles='openstreetmap',zoom_start=6)
    for index, row in df.iterrows():
        #icon=folium.Icon(color='purple',prefix='fa',icon='arrow-circle-down')
        icon=folium.features.CustomIcon(logo, icon_size=(34,34))
        marker = getMarker(row['lat'],row['lon'],row['name'], icon)

        marker.add_to(m5)
    
    return  m5

def marker_cluster(df, logo:str):

    '''
    This function creates a map where all the points are added, in the same way of the function marker plot,
    but it also aggregate into cluster closer points. This makes the map more good looking. 
    Moreover, by zooming in into the map the cluser become smaller and smaller until it the cluster is made by
    just one point.  
    '''

    m6=folium.Map(location=[41.9027835,12.4963655],tiles='openstreetmap',zoom_start=6)
    marker_cluster = MarkerCluster().add_to(m6)
    for index, row in df.iterrows():
        #icon=folium.Icon(color='purple',prefix='fa',icon='arrow-circle-down')
        icon=folium.features.CustomIcon(logo, icon_size=(34,34))
        message = '<strong>sezionD:'+ str(row['name'])
        #tip = message + '<br/>' + row['via']
        marker = getMarker(row['lat'],row['lon'],message, icon)
        #add to marker cluster 
        marker.add_to(marker_cluster)
    
    
    return m6

def heatmap_plot(df):

    '''
    This function creates a map where we can visualize the distribution of points via a 
    heatmap. Indeed the more the color is closer to red the higher is the concetration of points
    in that area.  
    '''

    m7 = folium.Map(location=[41.9027835,12.4963655],tiles='openstreetmap',zoom_start=6)
    data = df[['lat','lon']]
    HeatMap(data.values).add_to(m7)


    return m7


For instance, in the following chunk of code two maps concerning the italian factories in the italian territory are created: 
* the first is a map where factories are aggregated into clusters,
* the second is an heatmap showing the distribution of factories along the italian territory

In [ ]:
_df_ = read_csv_shp('D:\Geospatial_Project\Factories')
marker_cluster(_df_, logo = 'D:/Geospatial_Project/logo2.png')

In [ ]:
heatmap_plot(_df_) 

Now we do the same as before but for No-profit organizations in Italy.

In [ ]:
_df_ = read_csv_shp('D:/Geospatial_Project/No_Profit')
marker_cluster(_df_, logo = 'D:/Geospatial_Project/np2.jpg')


In [ ]:
heatmap_plot(_df_)

## Spatial Correlation

>  __Tobler's first law of geography:__  _Everything is related to everything else, but near things are more related than distant things (Tobler, 1970)_ 


In this section of the notebook we introduce methods of exploratory spatial data analysis that are intended to complement geovisualization through formal univariate and multivariate statistical tests for spatial autocorrelation.
The term _spatial auto-correlation_ refers to the presence of systematic spatial variation in a mapped variable. Where adjacent observations have similar data values, the map will show positive spatial autocorrelation. Where the adjacent observations have very contrasting values then the map will show negative spatial auto-correlation.
<br>
In this example, there will be shown the results relatively to the networth pro capite. 
<br>
NB: This measure has been chosen because ther is a high correaltion based on the loction of the territory. 

Now we define both the geodataframe at regional and provincial level. 
Then we will merge these two datasets. 

In [ ]:
Reg_Path = 'D:/Geospatial_Project/Dati/Limiti/Lim/Reg01012021/Reg01012021_WGS84.shp'
Prov_Path = 'D:/Geospatial_Project/Dati/Limiti/Lim/ProvCM01012021/ProvCM01012021_WGS84.shp'
# Reading the geodaframe

Reg_df = gpd.read_file(Reg_Path)
Prov_df = gpd.read_file(Prov_Path)


# Anomalous Regions names
Anomalous_Regions = ['Trentino-Alto Adige/Südtirol','Friuli-Venezia Giulia',"Valle d'Aosta/Vallée d'Aoste"]
#Sud_Sardinia_Agglomeration
Sud_Sardinia = ['Ogliastra','Olbia-Tempio','Medio Campidano','Carbonia-Iglesias']
# Macro_Areas 
Macro_Areas = ['Centro', 'Mezzogiorno', 'Nord']
# Italy 
Italy = ['Italia']
# Different_Names
Different_Names_BES = ['Bolzano/Bozen', 'Forlì-Cesena', 'Massa-Carrara', 'Reggio Calabria']
Deffirent_Names_ISTAT = ['Bolzano', "Forli'-Cesena", 'Massa Carrara', 'Reggio di Calabria']

# Read the BES_Statistics Dataframe
path_ = 'D:/Geospatial_Project/Nuovi_Dati2/'

In [ ]:
# Get the Istat regions 
Regions = Reg_df.DEN_REG.to_list()
# Get  BES Regions
Bes_Regions = regions_BES(Regions)
# Extract the set of all Bes provinces

In [ ]:
## PREPARE REG GEO DF ##
Reg_df = mod_col_geo(Reg_df)
Reg_df = order_df_regions(Reg_df, Bes_Regions)

In [ ]:
#df = read_dati_bes('Ambiente-Disponibilità di verde urbano-Totale-m2 per abitante.xlsx')
df = read_dati_bes(path_ + 'Benessere economico-Patrimonio pro capite-Totale-euro.xlsx')

In [ ]:
provinces = provinces_BES(df, Anomalous_Regions, Sud_Sardinia, Macro_Areas, Italy, Regions)

In [ ]:
## PREPARE PROV GEO DF ## 
Prov_df = clean_prov_geo(Prov_df, provinces)
# 'PROVINCES' #
# Create the provinces df
df_prov = order_df(df, provinces)
# Obtain the full geodataframe of provinces 
df_prov = from_df_to_gdf(df_prov, Prov_df)

In [ ]:
#Modify the crs in order to properly plot the geometries. 
df_prov.to_crs(4326, inplace = True)
Reg_df.to_crs(4326, inplace = True)

In [ ]:
df_prov2 = look_for_anomalies2(df_prov, 'V_2017')

In [ ]:
df_prov2 = df_prov2.reset_index()

In [ ]:
#joining the two dataframes.
prova_df = gpd.sjoin(Reg_df, 
                          df_prov2, how='inner', predicate='contains')

In [ ]:
prova_df2 = prova_df.reset_index()

In [ ]:
# Compute the average value by grouping provinces by their region (in 2017)
median_val = prova_df2['V_2017'].groupby([prova_df2['Reg']]).mean()

In [ ]:
#Visualizing the mean value of the networth pro-capite in the different regions. 
median_val

In [ ]:
#create a new aggregated dataframe
prova_df2 = prova_df.merge(median_val, on = 'Reg')

In [ ]:
prova_df2.head()

In [ ]:
#  create a simple choroplet map in order to have a first visualization of the data.
prova_df2.plot(column = 'V_2017_y')

In [ ]:
#reate a more sophisticated choroplet maps 
#that divides the region (based on the networth pro-capite in 2017) in 5 groups.
fig, ax = plt.subplots(figsize=(12,10), subplot_kw={'aspect':'equal'})
prova_df2.plot(column='V_2017_y', scheme='Quantiles', k=5, cmap='GnBu', legend=True, ax=ax)

__Spatial Autocorrelation:__

Visual inspection of the map pattern for the pro-capite networth allows us to search for spatial structure. If the spatial distribution of the networth was random, then we should not see any clustering of similar values on the map. However, our visual system is drawn to the lighter clusters in the south as well as the center, and a concentration of the darker hues (higher networth pro capite) in the north. 

Our brains are very powerful pattern recognition machines. However, sometimes they can be too powerful and lead us to detect false positives, or patterns where there are no statistical patterns. This is a particular concern when dealing with visualization of irregular polygons of differning sizes and shapes ([Rey,2022](https://sergerey.org/)) ([Wolf, 2022](https://www.ljwolf.org/posts/2020-sm2-lectures/)).

The concept of _spatial autocorrelation_ relates to the combination of two types of similarity: spatial similarity and attribute similarity. Although there are many different measures of spatial autocorrelation, they all combine these two types of simmilarity into a summary measure.

Let's use [PySAL](https://pysal.org/) to generate these two types of similarity measures.

__Spatial Similarity__
In spatial autocorrelation analysis, the spatial weights are used to formalize the notion of spatial similarity. As we have seen there are many ways to define spatial weights, here we will use queen contiguity:

In [ ]:
df = prova_df2
wq =  lps.weights.Queen.from_dataframe(df)
wq.transform = 'r'

Attribute Similarity
So the spatial weight between neighborhoods i and j indicates if the two are neighbors (i.e., geographically similar). What we also need is a measure of attribute similarity to pair up with this concept of spatial similarity. The spatial lag is a derived variable that accomplishes this for us. For neighborhood i the spatial lag is defined as:
<br>
                                   <h3><center>$ylag_{i}=\sum_j{w_{i,j}y_{j}}$</center></h3>



In [ ]:
y = df['V_2017_y']
ylag = lps.weights.lag_spatial(wq, y)

In [ ]:
import mapclassify as mc
ylagq5 = mc.Quantiles(ylag, k=5)

In [ ]:
f, ax = plt.subplots(1, figsize=(9, 9))
df.assign(cl=ylagq5.yb).plot(column='cl', categorical=True, \
        k=5, cmap='GnBu', linewidth=0.1, ax=ax, \
        edgecolor='white', legend=True)
ax.set_axis_off()
plt.title("Spatial Lag Median Networth pro-capite (Quintiles)")

plt.show()

The quintile map for the spatial lag tends to enhance the impression of value similarity in space. It is, in effect, a local smoother

In [ ]:
df['lag_median_val'] = ylag
f,ax = plt.subplots(1,2,figsize=(2.16*4,4))
df.plot(column='V_2017_y', ax=ax[0], edgecolor='k',
        scheme="quantiles",  k=5, cmap='GnBu')
ax[0].axis(df.total_bounds[np.asarray([0,2,1,3])])
ax[0].set_title("Networth pro-capite")
df.plot(column='lag_median_val', ax=ax[1], edgecolor='k',
        scheme='quantiles', cmap='GnBu', k=5)
ax[1].axis(df.total_bounds[np.asarray([0,2,1,3])])
ax[1].set_title("Spatial Lag Neworth pro-capite")
ax[0].axis('off')
ax[1].axis('off')
plt.show()

However, we still have the challenge of visually associating the value of the networth in a region with the value of the spatial lag of values for the focal unit. The latter is a weighted average of list networths in the focal region.

To complement the geovisualization of these associations we can turn to formal statistical measures of spatial autocorrelation.

__Global Spatial Autocorrelation__
We begin with a simple case where the variable under consideration is binary. This is useful to unpack the logic of spatial autocorrelation tests. So even though our attribute is a continuously valued one, we will convert it to a binary case to illustrate the key concepts:

In [ ]:
y.median()

In [ ]:
yb = y > y.median()
sum(yb)

In [ ]:
yb = y > y.median()
labels = ["0 Low", "1 High"]
yb = [labels[i] for i in 1*yb] 
df['yb'] = yb

In [ ]:
fig, ax = plt.subplots(figsize=(12,10), subplot_kw={'aspect':'equal'})
df.plot(column='yb', cmap='binary', edgecolor='grey', legend=True, ax=ax)

__Join counts__
One way to formalize a test for spatial autocorrelation in a binary attribute is to consider the so-called _joins_. A join exists for each neighbor pair of observations, and the joins are reflected in our binary spatial weights object wq.

Each unit can take on one of two values "Black" or "White", and so for a given pair of neighboring locations there are three different types of joins that can arisD:

Black Black (BB)
White White (WW)
Black White (or White Black) (BW)
Given that we have 47 Black polygons on our map, what is the number of Black Black (BB) joins we could expect if the process were such that the Black polygons were randomly assigned on the map? This is the logic of join count statistics.

We can use the ´esda´ package from ´PySAL´ to carry out join count analysis:

In [ ]:
import esda 
yb = 1 * (y > y.median()) # convert back to binary
wq =  lps.weights.Queen.from_dataframe(df)
wq.transform = 'b'
np.random.seed(12345)
jc = esda.join_counts.Join_Counts(yb, wq)

In [ ]:
# Visualize the results of the join analysis
import seaborn as sbn
sbn.kdeplot(jc.sim_bb, shade=True)
plt.vlines(jc.bb, 0, 0.075, color='r')
plt.vlines(jc.mean_bb, 0,0.075)
plt.xlabel('BB Counts')

The density portrays the distribution of the BB counts, with the black vertical line indicating the mean BB count from the synthetic realizations and the red line the observed BB count for the networth procapite. Clearly our observed value is extremely high. A pseudo p-value summarizes this:

In [ ]:
# look at the pseudo p-value
jc.p_sim_bb 

Since this is below conventional significance levels, we would reject the null of complete spatial randomness in favor of spatial autocorrelation in networth pro-capite.



__Continuous Case__
The join count analysis is based on a binary attribute, which can cover many interesting empirical applications where one is interested in presence and absence type phenomena. In our case, we artificially created the binary variable, and in the process we throw away a lot of information in our originally continuous attribute. Turning back to the original variable, we can explore other tests for spatial autocorrelation for the continuous case.

First, we transform our weights to be row-standardized, from the current binary state:

In [ ]:
wq.transform = 'r'
y = df['V_2017_y']

In order to assess whether spatial auto-correlation is statistically significant, it is necessary to perfrom the Moran I statistic. In essence, it is a cross-product statistic between a variable and its spatial lag, with the variable expressed in deviations from its mean. For an observation, at location i, this is expressed as

$z_i=x_i − \bar{x}$, where $\bar{x}$ is the mean of variable $x$.

Moran’s I statistic is then:
$ I = \frac{{}\sum_{i} \sum_{j} w_{ij} * z_i * z_j /{S_0}}{\sum_{i} z_{i}^2 /{n}}$


with $w_{ij}$ as the elements of the spatial weights matrix, $S_0 = \sum_i \sum_j w_{ij}$ as the sum of all the weights, and $n$ as the number of observations. 

Inference for Moran’s I is based on a null hypothesis of spatial randomness. The distribution of the statistic under the null can be derived using either an assumption of normality (independent normal random variates), or so-called randomization (i.e., each value is equally likely to occur at any location). While the analytical derivations provide easy to interpret expressions for the mean and the variance of the statistic under the null hypothesis, inference based on them employs an approximation to a standard normal distribution, which may be inappropriate when the underlying assumptions are not satisfied

Moran's I is a test for global autocorrelation for a continuous attribute:

In [ ]:
np.random.seed(12345)
mi = esda.moran.Moran(y, wq)
mi.I

Again, our value for the statistic needs to be interpreted against a reference distribution under the null of CSR. PySAL uses a similar approach as we saw in the join count analysis: random spatial permutations.

In [ ]:
sbn.kdeplot(mi.sim, shade=True)
plt.vlines(mi.I, 0, 1, color='r')
plt.vlines(mi.EI, 0,1)
plt.xlabel("Moran's I")

In [ ]:
mi.p_sim

__Local Autocorrelation: Hot Spots, Cold Spots, and Spatial Outliers__
In addition to the Global autocorrelation statistics, PySAL has many local autocorrelation statistics. Let's compute a local Moran statistic for the same d


In [ ]:
# set a random seed
np.random.seed(12345)

In [ ]:
wq.transform = 'r'
lag_price = lps.weights.lag_spatial(wq, df['V_2017_y'])

The Moran scatter plot consists of a plot with the spatially lagged variable on the y-axis and the original variable on the x-axis. The slope of the linear fit to the scatter plot equal Moran's I. A variable $z$ is considered, given in deviations from the mean. With row-standardized weights, the sum of all the weights ($S_0$) equals the number of observations ($n$). As a result, the expression for Moran's I simplifies to:
$I =  \frac{{}\sum_{i} \sum_{j} w_{ij} * z_i * z_j /{S_0}}{\sum_{i} z_{i}^2 /{n}} =  \frac{\sum_{i} (z_i X \sum_j w_{ij}z_j)} {\sum_i z_i^2}$
This turns out to be the slope of a regression of $\sum_j w_{ij}z_j$ on $z_i$. This is the principle underlying the Moran scatter plot.

An important aspect of the visualization in the Moran scatter plot is the classification of the nature of spatial auto-correlation into four categories. Since the plot is centered on the mean (of zero), all points to the right of the mean have $z_i>0$ and all points to the left have $z_i<0$. We refer to these values respectively as high and low, in the limited sense of higher or lower than average. Similarly, we can classify the values for the spatial lag above and below the mean as high and low.

The scatter plot is then easily decomposed into four quadrants. The upper-right quadrant and the lower-left quadrant correspond with positive spatial auto-correlation (similar values at neighboring locations). We refer to them as respectively high-high and low-low spatial auto-correlation. In contrast, the lower-right and upper-left quadrant correspond to negative spatial auto-correlation (dissimilar values at neighboring locations). We refer to them as respectively high-low and low-high spatial auto-correlation.

The classification of the spatial auto-correlation into four types begins to make the connection between global and local spatial auto-correlation. However, it is important to keep in mind that the classification as such does not imply significance. This is further explored in our discussion of local indicators of spatial association (LISA).

In [ ]:
price = df['V_2017_y']
b, a = np.polyfit(price, lag_price, 1)
f, ax = plt.subplots(1, figsize=(9, 9))

plt.plot(price, lag_price, '.', color='firebrick')

 # dashed vert at mean of the price
plt.vlines(price.mean(), lag_price.min(), lag_price.max(), linestyle='--')
 # dashed horizontal at mean of lagged price 
plt.hlines(lag_price.mean(), price.min(), price.max(), linestyle='--')

# red line of best fit using global I as slope
plt.plot(price, a + b*price, 'r')
plt.title('Moran Scatterplot')
plt.ylabel('Spatial Lag of networth pro-capite')
plt.xlabel('Networth pro-capite')
plt.show()

Now, instead of a single I statistic, we have an array of local Ii statistics, stored in the ´.Is´ attribute, and p-values from the simulation are ´in p_sim´.

In [ ]:
li = esda.moran.Moran_Local(y, wq)

We can distinguish the specific type of local spatial association reflected in the four quadrants of the Moran Scatterplot abovD:

In [ ]:
sig = li.p_sim < 0.05
hotspot = sig * li.q==1
coldspot = sig * li.q==3
doughnut = sig * li.q==2
diamond = sig * li.q==4

In [ ]:
spots = ['n.sig.', 'hot spot']
labels = [spots[i] for i in hotspot*1]

In [ ]:
df = df
from matplotlib import colors
hmap = colors.ListedColormap(['red', 'lightgrey'])
f, ax = plt.subplots(1, figsize=(9, 9))
df.assign(cl=labels).plot(column='cl', categorical=True, \
        k=2, cmap=hmap, linewidth=0.1, ax=ax, \
        edgecolor='white', legend=True)
ax.set_axis_off()
plt.show()

In [ ]:
spots = ['n.sig.', 'cold spot']
labels = [spots[i] for i in coldspot*1]

In [ ]:
df = df
from matplotlib import colors
hmap = colors.ListedColormap(['blue', 'lightgrey'])
f, ax = plt.subplots(1, figsize=(9, 9))
df.assign(cl=labels).plot(column='cl', categorical=True, \
        k=2, cmap=hmap, linewidth=0.1, ax=ax, \
        edgecolor='white', legend=True)
ax.set_axis_off()
plt.show()

In [ ]:
sig = 1 * (li.p_sim < 0.05)
hotspot = 1 * (sig * li.q==1)
coldspot = 3 * (sig * li.q==3)
doughnut = 2 * (sig * li.q==2)
diamond = 4 * (sig * li.q==4)
spots = hotspot + coldspot + doughnut + diamond
spots

In [ ]:
spot_labels = [ '0 ns', '1 hot spot', '2 doughnut', '3 cold spot', '4 diamond']
labels = [spot_labels[i] for i in spots]

In [ ]:
from matplotlib import colors
hmap = colors.ListedColormap([ 'lightgrey', 'red', 'lightblue', 'blue', 'pink'])
f, ax = plt.subplots(1, figsize=(9, 9))
df.assign(cl=labels).plot(column='cl', categorical=True, \
        k=2, cmap=hmap, linewidth=0.1, ax=ax, \
        edgecolor='white', legend=True)
ax.set_axis_off()
plt.show()

As we can see from this last plot, there is a clear division among the south and the north of Italy for what concerns the net woth pro-capita: Northern regions habitants have an higher net worth pro-capita than the ones of southern regions.

## Isochrone map

Although we have many data for each indicator, the data related to the _Subjective wellbeing_ are totally missing. 
<br>
According to the [BES](https://www.istat.it/it/files/2018/04/12-domains-scientific-commission.pdf) this indicator relies on four sub-indicators:
1) Life satisfaction
2) Leaisure time satisfaction 
3) Positive judgments for future perspective
4) Negative judgments for future perspecitve
<br>


It is quite difficult to find this kind of data without having the results of ad-hoc surveys constructed for this specific aim. Moreover, it is also difficult to find some other index/indicators that can approximate them especially if we work with geodata. 
<br>

However, could be interesting to see how many amenties can be reached in 10-20minutes (isochrone maps) from the city center. Indeed, having this kind of representation could help us to understand something about the second sub-indicator: Leisure time satisfaction. As a matter of fact, having many pubs, cinema and other places where it is possible to spend your free time can be positively correlated to have a high leisure-time satisfaction. 



In [ ]:
! pip install openrouteservice

In [ ]:
# import dependencies 
import openrouteservice as ors
import warnings
warnings.simplefilter("ignore")

In the following chunk of code we take advantage of the geocoder to extract the representative points of the 10 biggest italian cities.
<br>
NB: we consider just the first 10 cities to obtain a clearer map. For instance, if we decided to collect the amenities for all the provinces of italy, then the map would have been too messy.  

In [ ]:
cols = ['city']
names = [('Roma'),('Torino'),('Genova'),('Bari'), ('Milano'), ('Genova'), ('Palermo'), ('Napoli'), ('Bologna'), ('Verona')]
cities = gpd.GeoDataFrame(names,columns=cols)
geo_cities = gpd.tools.geocode(cities.city, provider="arcgis")

As did before, we use osm to extract the location of all the amanities related to freetime activities and entratainment in the required places. 
Then through the already mentioned function ´read_csv_shp´ we create a dataframe with the coordinates and names of these facilities. 

In [ ]:
complete_df = read_csv_shp('Freetime')

Then we firstly create a folium marker cluster map

In [ ]:
m5 = marker_cluster(complete_df, 'D:\Geospatial_Project\pointer.png')

In the following chunk of code a customized pointer with the logo of each city is added to the map.

In [ ]:
for index, row in geo_cities.iterrows():
      icon=folium.features.CustomIcon(os.getcwd() + '/Icons/' + row['address']+ '.png',icon_size=(40,40))
marker = getMarker(row['geometry'].centroid.y,row['geometry'].centroid.x, row['address'],row['address'], icon)
marker.add_to(m5)

Then we had the isochrone maps that highlight the facilities that can be reached in 10-20minutes of walk having as starting point the representative point of the city.

In [ ]:
ors_key = '5b3ce3597851110001cf6248d3c5776fac4c4918b5258bb3659ddec6'
client = ors.Client(key=ors_key)

In [ ]:
for i in range(len(geo_cities)):
  city = geo_cities.iloc[i]
  coordinate = [[city.geometry.centroid.x, city.geometry.centroid.y]]
  iscorhrone = client.isochrones( locations=coordinate,
    profile='foot-walking',
    range=[600, 1200],
    validate=False,)
  popup_message = f'outline shows areas reachable within 10-20 minutes'
  folium.GeoJson(iscorhrone, name='isochrone', tooltip= popup_message).add_to(m5)

In [ ]:
# Visualize the map
m5

## Best Territories

The last stage of the analysis is trying to understand which are the best areas for each of the 12 subindicators and also the best territory over all. 
<br>
To do a simple model has been adopted: 
1) Firstly, has been created a dictionary that has as keys all the subindicators and as value  it has ´True´ when the higher the value of the subindicator the better it is (e.g. amount of energy coming from renewable sources) and ´False´ when the lower the value the better it is. 
2) Secondly, for each level of analysis (Provinces, Regions, and Macroareas), a dictionary that will store the results has been createed. These dictionaries have as key the territories' name and as value another dictionary. This second dictionary has as keys the twelve subindicators and as values an empty list.
3) For each level of analysis and for all indicator, the territories are sorted according to the measure (in asceding order if the subindicator is positive and on contrary in descending order). 
4) A rank is assigned to each territory based on the order. 
5) All these rankings are stored in the subdictionary
6) For each indicator, the best territories are the ones whose sum of the subindicators' raking is the lower
7) The overall best territories are the ones which have the lowest sum of rakings among all subindicators. 

NB: The rakings are based on the most recent data available. 

NB: This way of assessing the best territories is obviously an extreme approximation of the reality since we just consider the subindicators and we consider them in an additive way, without weighting the importance of them. 



In [ ]:
# This dictionary tells us whether it is better to have an higher value for the indicator or not.

diz_sub = {'Benessere economico-Importo medio annuo pro_capite dei redditi pensionistici-Femmine-euro.xlsx': True,
 'Benessere economico-Importo medio annuo pro_capite dei redditi pensionistici-Maschi-euro.xlsx': True,
 'Benessere economico-Importo medio annuo pro_capite dei redditi pensionistici-Totale-euro.xlsx': True,
 'Benessere economico-Patrimonio pro capite-Totale-euro.xlsx': True,
 'Benessere economico-Pensionati con pensione di basso importo-Femmine-valori percentuali.xlsx': False,
 'Benessere economico-Pensionati con pensione di basso importo-Maschi-valori percentuali.xlsx': False,
 'Benessere economico-Pensionati con pensione di basso importo-Totale-valori percentuali.xlsx': False,
 'Benessere economico-Reddito medio disponibile pro capite-Totale-euro.xlsx': True,
 'Benessere economico-Retribuzione media annua dei lavoratori dipendenti-Femmine-euro.xlsx': True,
 'Benessere economico-Retribuzione media annua dei lavoratori dipendenti-Maschi-euro.xlsx': True,
 'Benessere economico-Retribuzione media annua dei lavoratori dipendenti-Totale-euro.xlsx': True,
 'Benessere economico-Tasso di ingresso in sofferenza dei prestiti bancari alle famiglie-Totale-valori percentuali.xlsx': False,
 'Istruzione e formazione-Competenza alfabetica non adeguata-Femmine-valori percentuali.xlsx': False,
 'Istruzione e formazione-Competenza alfabetica non adeguata-Maschi-valori percentuali.xlsx': False,
 'Istruzione e formazione-Competenza alfabetica non adeguata-Totale-valori percentuali.xlsx': False,
 'Istruzione e formazione-Competenza numerica non adeguata-Femmine-valori percentuali.xlsx': False,
 'Istruzione e formazione-Competenza numerica non adeguata-Maschi-valori percentuali.xlsx': False,
 'Istruzione e formazione-Competenza numerica non adeguata-Totale-valori percentuali.xlsx': False,
 'Istruzione e formazione-Giovani che non lavorano e non studiano (Neet)-Totale-valori percentuali.xlsx': False,
 'Istruzione e formazione-Laureati e altri titoli terziari (25-39 anni)-Totale-valori percentuali.xlsx': True,
 'Istruzione e formazione-Partecipazione alla formazione continua-Totale-valori percentuali.xlsx': True,
 "Istruzione e formazione-Passaggio all'università-Totale-valori percentuali (tasso specifico di coorte).xlsx": True,
 'Istruzione e formazione-Persone con almeno il diploma (25-64 anni)-Totale-valori percentuali.xlsx': True,
 'Ambiente-Conferimento dei rifiuti urbani in discarica-Totale-valori percentuali.xlsx': False,
 'Ambiente-Dispersione da rete idrica comunale-Totale-valori percentuali.xlsx': False,
 'Ambiente-Disponibilità di verde urbano-Totale-m2 per abitante.xlsx': True,
 'Ambiente-Energia elettrica da fonti rinnovabili-Totale-valori percentuali.xlsx': True,
 'Ambiente-Impermeabilizzazione del suolo da copertura artificiale-Totale-valori percentuali.xlsx': True,
 "Ambiente-Qualità dell'aria urbana - Biossido di azoto-Totale-valori percentuali.xlsx": False,
 "Ambiente-Qualità dell'aria urbana - PM10-Totale-valori percentuali.xlsx": False,
 'Ambiente-Raccolta differenziata dei rifiuti urbani-Totale-valori percentuali.xlsx': True,
 'Salute-Mortalità infantile-Totale-per 1.000 nati vivi.xlsx': False,
 'Salute-Mortalità per demenze e malattie del sistema nervoso (65 anni e più)-Femmine-tassi standardizzati per 10.000 residenti.xlsx': False,
 'Salute-Mortalità per demenze e malattie del sistema nervoso (65 anni e più)-Maschi-tassi standardizzati per 10.000 residenti.xlsx': False,
 'Salute-Mortalità per demenze e malattie del sistema nervoso (65 anni e più)-Totale-tassi standardizzati per 10.000 residenti.xlsx': False,
 'Salute-Mortalità per incidenti stradali (15-34 anni)-Femmine-tassi standardizzati per 10.000 residenti.xlsx': False,
 'Salute-Mortalità per incidenti stradali (15-34 anni)-Maschi-tassi standardizzati per 10.000 residenti.xlsx': False,
 'Salute-Mortalità per incidenti stradali (15-34 anni)-Totale-tassi standardizzati per 10.000 residenti.xlsx': False,
 'Salute-Mortalità per tumore (20-64 anni)-Femmine-tassi standardizzati per 10.000 residenti.xlsx': False,
 'Salute-Mortalità per tumore (20-64 anni)-Maschi-tassi standardizzati per 10.000 residenti.xlsx': False,
 'Salute-Mortalità per tumore (20-64 anni)-Totale-tassi standardizzati per 10.000 residenti.xlsx': False,
 'Salute-Speranza di vita alla nascita-Femmine-numero medio di anni.xlsx': True,
 'Salute-Speranza di vita alla nascita-Maschi-numero medio di anni.xlsx': True,
 'Salute-Speranza di vita alla nascita-Totale-numero medio di anni.xlsx': True,
 'Innovazione, ricerca e creatività-Addetti nelle imprese culturali-Totale-valori percentuali.xlsx': True,
 'Innovazione, ricerca e creatività-Mobilità dei laureati italiani (25-39 anni)-Totale-per 1.000 laureati residenti.xlsx': True,
 'Innovazione, ricerca e creatività-Propensione alla brevettazione-Totale-per milioni di abitanti.xlsx': True,
 'Paesaggio e patrimonio culturale-Densità di verde storico-Totale-per 100 m2.xlsx': True,
 'Paesaggio e patrimonio culturale-Densità e rilevanza del patrimonio museale-Totale-per 100 km2.xlsx': True,
 'Paesaggio e patrimonio culturale-Diffusione delle aziende agrituristiche-Totale-per 100 km2.xlsx': True,
 'Politica e istituzioni-Affollamento degli istituti di pena-Totale-valori percentuali.xlsx': False,
 'Politica e istituzioni-Amministratori comunali con meno di 40 anni-Totale-valori percentuali.xlsx': True,
 'Politica e istituzioni-Amministratori comunali donne-Totale-valori percentuali.xlsx': True,
 'Politica e istituzioni-Partecipazione elettorale (elezioni regionali)-Totale-valori percentuali.xlsx': True,
 'Politica e istituzioni-Partecipazione elettorale-Totale-valori percentuali.xlsx': True,
 "Qualità dei servizi-Bambini che hanno usufruito dei servizi comunali per l'infanzia-Totale-valori percentuali.xlsx": True,
 'Qualità dei servizi-Emigrazione ospedaliera in altra regione-Totale-valori percentuali.xlsx': False,
 'Qualità dei servizi-Irregolarità del servizio elettrico-Totale-numero medio per utente.xlsx': False,
 'Qualità dei servizi-Posti km offerti dal Tpl-Totale-valori per abitante.xlsx': True,
 'Sicurezza-Altri delitti violenti denunciati-Totale-per 10.000 abitanti.xlsx': False,
 'Sicurezza-Delitti diffusi denunciati-Totale-per 10.000 abitanti.xlsx': False,
 'Sicurezza-Mortalità stradale in ambito extraurbano-Totale-valori percentuali.xlsx': False,
 'Sicurezza-Omicidi-Totale-per 100.000 abitanti.xlsx': False,
 'Relazioni sociali-Organizzazioni non profit-Totale-per 10.000 abitanti.xlsx': True,
 'Relazioni sociali-Scuole accessibili-Totale-valori percentuali.xlsx': True,
 "Lavoro e conciliazione dei tempi di vita-Giornate retribuite nell'anno (lavoratori dipendenti)-Totale-valori percentuali.xlsx": True,
 'Lavoro e conciliazione dei tempi di vita-Tasso di infortuni mortali e inabilità permanente-Femmine-per 10.000 occupati.xlsx': False,
 'Lavoro e conciliazione dei tempi di vita-Tasso di infortuni mortali e inabilità permanente-Maschi-per 10.000 occupati.xlsx': False,
 'Lavoro e conciliazione dei tempi di vita-Tasso di infortuni mortali e inabilità permanente-Totale-per 10.000 occupati.xlsx': False,
 'Lavoro e conciliazione dei tempi di vita-Tasso di mancata partecipazione al lavoro giovanile (15-29 anni)-Femmine-valori percentuali.xlsx': False,
 'Lavoro e conciliazione dei tempi di vita-Tasso di mancata partecipazione al lavoro giovanile (15-29 anni)-Maschi-valori percentuali.xlsx': False,
 'Lavoro e conciliazione dei tempi di vita-Tasso di mancata partecipazione al lavoro giovanile (15-29 anni)-Totale-valori percentuali.xlsx': False,
 'Lavoro e conciliazione dei tempi di vita-Tasso di mancata partecipazione al lavoro-Femmine-valori percentuali.xlsx': False,
 'Lavoro e conciliazione dei tempi di vita-Tasso di mancata partecipazione al lavoro-Maschi-valori percentuali.xlsx': False,
 "Lavoro e conciliazione dei tempi di vita-Giornate retribuite nell'anno (lavoratori dipendenti)-Femmine-valori percentuali.xlsx": True,
 "Lavoro e conciliazione dei tempi di vita-Giornate retribuite nell'anno (lavoratori dipendenti)-Maschi-valori percentuali.xlsx": True,
 'Lavoro e conciliazione dei tempi di vita-Tasso di mancata partecipazione al lavoro-Totale-valori percentuali.xlsx': False,
 'Lavoro e conciliazione dei tempi di vita-Tasso di occupazione (20-64 anni)-Femmine-valori percentuali.xlsx': True,
 'Lavoro e conciliazione dei tempi di vita-Tasso di occupazione (20-64 anni)-Maschi-valori percentuali.xlsx': True,
 'Lavoro e conciliazione dei tempi di vita-Tasso di occupazione (20-64 anni)-Totale-valori percentuali.xlsx': True,
 'Lavoro e conciliazione dei tempi di vita-Tasso di occupazione giovanile (15-29 anni)-Femmine-valori percentuali.xlsx': True,
 'Lavoro e conciliazione dei tempi di vita-Tasso di occupazione giovanile (15-29 anni)-Maschi-valori percentuali.xlsx': True,
 'Lavoro e conciliazione dei tempi di vita-Tasso di occupazione giovanile (15-29 anni)-Totale-valori percentuali.xlsx': True}

### Create the dictionaries

#### Provinces

Create the dictionary where all the rakings will be stored.

In [ ]:
diz_prov = dict()
for prov in provinces:
    classes = os.listdir('D:\Geospatial_Project\Dati_Streamlite')
    classes.remove('Wk_2')
    classes.remove('Subjective well-being')
    diz_ind = dict()
    for folder in classes:
        diz_ind[folder] = []
                
    diz_prov[prov] = diz_ind 

Collect the rankings.

NB: This operation needs to be repeateed for each indicator.

In [ ]:
years = []

for i in range(4,20):
    if len(str(i)) == 1: 
        data = 'V_200' + str(i)
    else:
        data = 'V_20' + str(i) 

    years.append(data)

years.reverse()

In [ ]:
dir = 'D:\Geospatial_Project\Dati_Streamlite\\'
ind = 'Work & Life Balance'
for sub_ind in os.listdir(dir+ind):
    # Read the geodataframe
    Prov_df = gpd.read_file(Prov_Path)
    df = read_dati_bes(dir+ind+ '\\' + sub_ind)
    ## PREPARE PROV GEO DF ## 
    Prov_df = clean_prov_geo(Prov_df, provinces)
    # Create the provinces df
    df_prov = order_df(df, provinces)
    # Obtain the full geodataframe of provinces 
    df_prov = from_df_to_gdf(df_prov, Prov_df)
    # reset the index
    df_prov.reset_index(inplace = True) 
    # check the data for the most recent year
    for y_ in years: 
        if not df_prov[y_].isna().all():
            break
    
    
    df_prov = look_for_anomalies2(df_prov, y_)
    if diz_sub[sub_ind]: # check whether having an high value is good or not 
        df_prov.sort_values(by = y_, inplace= True, ascending= False)
    else:
        df_prov.sort_values(by = y_, inplace = True)
    
    df_prov['Rank'] = list(range(1, len(df_prov)+1))

    for i in range(len(df_prov)):
        territorio = df_prov.iloc[i]['TERRITORIO']
        rank = df_prov.iloc[i]['Rank']
        diz_prov[territorio][ind].append(rank)

    # MISSING VALUES
    j = 1
    missing_ter = set(provinces) - set(df_prov.TERRITORIO.to_list())
    for mt in missing_ter:
        #print(mt)
        diz_prov[mt][ind].append(len(df_prov)+j)
        j += 1 

#### Regions

Create the dictionary for the rankings

In [ ]:
diz_reg = dict()
for reg in Bes_Regions:
    classes = os.listdir('D:\Geospatial_Project\Dati_Streamlite')
    classes.remove('Wk_2')
    classes.remove('Subjective well-being')
    diz_ind = dict()
    for folder in classes:
        diz_ind[folder] = []
                
    diz_reg[reg] = diz_ind 

Store the rankings into the dictionary

NB: This operation needs to be repeated for all the indicators

In [ ]:
dir = 'D:\Geospatial_Project\Dati_Streamlite\\'
ind = 'Economic well-being'
for sub_ind in os.listdir(dir+ind):
    # Read the geodataframe
    Reg_df = gpd.read_file(Reg_Path)
    df = read_dati_bes(dir+ind+ '\\' + sub_ind)
    ## PREPARE REG GEO DF ## 
    Reg_df = mod_col_geo(Reg_df)
    Reg_df = order_df_regions(Reg_df, Bes_Regions)
    # Creating Regions DataFrame
    df_reg = order_df(df, Bes_Regions)
    # Create the geodaframe used in the representation
    df_reg = from_df_to_gdf(df_reg, Reg_df)
    # reset the index
    df_reg.reset_index(inplace = True) 
    # check the data for the most recent year
    for y_ in years: 
        if not df_reg[y_].isna().all():
            break
    
    
    df_reg = look_for_anomalies2(df_reg, y_)
    if diz_sub[sub_ind]: # check whether having an high value is good or not 
        df_reg.sort_values(by = y_, inplace= True, ascending= False)
    else:
        df_reg.sort_values(by = y_, inplace = True)
    
    df_reg['Rank'] = list(range(1, len(df_reg)+1))

    for i in range(len(df_reg)):
        territorio = df_reg.iloc[i]['TERRITORIO']
        rank = df_reg.iloc[i]['Rank']
        diz_reg[territorio][ind].append(rank)

    # MISSING VALUES
    j = 1
    missing_ter = set(Bes_Regions) - set(df_reg.TERRITORIO.to_list())
    for mt in missing_ter:
        diz_reg[mt][ind].append(len(df_reg)+j)
        j += 1 

#### Macroareas

Create the dictionary 

In [ ]:
diz_mac = dict()
for mac in Macro_Areas:
    classes = os.listdir('D:\Geospatial_Project\Dati_Streamlite')
    classes.remove('Wk_2')
    classes.remove('Subjective well-being')
    diz_ind = dict()
    for folder in classes:
        diz_ind[folder] = []
                
    diz_mac[mac] = diz_ind 

Store the rankings into the dictionary

NB: this operation needs to be repeated for all the indicators

In [ ]:
dir = 'D:\Geospatial_Project\Dati_Streamlite\\'
ind = 'Work & Life Balance'
for sub_ind in os.listdir(dir+ind):
    # Read the geodataframe
    Macro_df = gpd.read_file(Macro_Path)
    df = read_dati_bes(dir+ind+ '\\' + sub_ind) 
    ## PREPARE MACRO GEO DF ##
    Macro_df = aggregate_macros(Macro_df)
    # Creating Macro DataFrame
    df_macro = order_df(df, Macro_Areas)
    # Create the geodaframe used in the representation
    df_macro = from_df_to_gdf(df_macro, Macro_df)
    # reset the index
    df_macro.reset_index(inplace = True) 
    # check the data for the most recent year
    for y_ in years: 
        if not df_macro[y_].isna().all():
            break
    
    
    df_macro = look_for_anomalies2(df_macro, y_)
    if diz_sub[sub_ind]: # check whether having an high value is good or not 
        df_macro.sort_values(by = y_, inplace= True, ascending= False)
    else:
        df_macro.sort_values(by = y_, inplace = True)
    
    df_macro['Rank'] = list(range(1, len(df_macro)+1))

    for i in range(len(df_macro)):
        territorio = df_macro.iloc[i]['TERRITORIO']
        rank = df_macro.iloc[i]['Rank']
        diz_mac[territorio][ind].append(rank)

    # MISSING VALUES
    j = 1
    missing_ter = set(Macro_Areas) - set(df_macro.TERRITORIO.to_list())
    for mt in missing_ter:
        diz_mac[mt][ind].append(len(df_macro)+j)
        j += 1 

### Create Dataframes

In order to skip the above computations, it is possible to save these three dictionaries via ´pickle´

In [ ]:
import pickle

#### Save the dictionaries

In [ ]:
with open('D:\Geospatial_Project\Dictionaries\diz_prov.pkl', 'wb') as f:
    pickle.dump(diz_prov, f)

In [ ]:
with open('D:\Geospatial_Project\Dictionaries\diz_reg.pkl', 'wb') as f:
    pickle.dump(diz_reg, f)

In [ ]:
with open('D:\Geospatial_Project\Dictionaries\diz_mac.pkl', 'wb') as f:
    pickle.dump(diz_mac, f)

#### Load the dictionaries

In [ ]:
with open('D:\Geospatial_Project\Dictionaries\diz_mac.pkl', 'rb') as f:
    prova_mac = pickle.load(f)

In [ ]:
with open('D:\Geospatial_Project\Dictionaries\diz_reg.pkl', 'rb') as f:
    prova_reg = pickle.load(f)

In [ ]:
with open('D:\Geospatial_Project\Dictionaries\diz_prov.pkl', 'rb') as f:
    prova_prov = pickle.load(f)

In [ ]:
indicatori = os.listdir('D:\Geospatial_Project\Dati_Streamlite')
indicatori.append('TERRITORIO')
indicatori.remove('Subjective well-being')

In [ ]:
def df_ranking(prova_mac, indicatori):
    ranking = pd.DataFrame(columns=indicatori)
    row = []
    for territorio in prova_mac:
        for ind in prova_mac[territorio]:
            row.append(sum(prova_mac[territorio][ind]))

        row.append(territorio)    
        series = pd.Series(row, index = ranking.columns)
        ranking = ranking.append(series, ignore_index= True)

        row = []
    ranking.set_index('TERRITORIO', inplace = True)
    return ranking

#### Macroareas

In [ ]:
ranking = df_ranking(prova_mac, indicatori)

Macro_df = gpd.read_file(Macro_Path)
Macro_df = aggregate_macros(Macro_df)

In [ ]:
ranking.set_index('TERRITORIO', inplace = True)
ranking = from_df_to_gdf(ranking, Macro_df)
ranking.reset_index(inplace= True)
ranking.sort_values(by = 'Environment', inplace = True) # we have chosen environment as example

__Create the map__

In [ ]:
m5=folium.Map(location=[41.9027835,12.4963655],tiles='openstreetmap',zoom_start=6)
j = 1 
for index, row in ranking.iterrows():
  icon=folium.features.CustomIcon('D:/Geospatial_Project/Medaglie/' + str(j) + '.png' ,icon_size=(40,40))
  marker = getMarker(row['geometry'].centroid.y,row['geometry'].centroid.x, row['TERRITORIO'], icon)
  marker.add_to(m5)
  j += 1

#### Region

In [ ]:
Reg_df = mod_col_geo(Reg_df)
Reg_df = order_df_regions(Reg_df, Bes_Regions)

In [ ]:
ranking = df_ranking(prova_reg, indicatori)
ranking = from_df_to_gdf(ranking, Reg_df)
ranking.reset_index(inplace = True)
ranking.sort_values(by = ['Health'], inplace = True)

In [ ]:
ranking2 = ranking.iloc[:3] #we care about the first three
ranking2.to_crs(4326, inplace = True)

In [ ]:
m6=folium.Map(location=[41.9027835,12.4963655],tiles='openstreetmap',zoom_start=6)
j = 1 
for index, row in ranking2.iterrows():
  icon=folium.features.CustomIcon('D:/Geospatial_Project/Medaglie/' + str(j) + '.png' ,icon_size=(40,40))
  marker = getMarker(row['geometry'].centroid.y,row['geometry'].centroid.x, row['TERRITORIO'], icon)
  marker.add_to(m6)
  j += 1

#### Provinces

In [ ]:
ranking = df_ranking(prova_prov, indicatori)

In [ ]:
Prov_df = gpd.read_file(Prov_Path)
Prov_df = clean_prov_geo(Prov_df, provinces)

In [ ]:
ranking = from_df_to_gdf(ranking, Prov_df)
ranking.reset_index(inplace= True)
ranking.sort_values(by = 'Environment', inplace = True)

In [ ]:
ranking2 = ranking.iloc[:3] #we care about the first three
ranking2.to_crs(4326, inplace = True)

In [ ]:
m7=folium.Map(location=[41.9027835,12.4963655],tiles='openstreetmap',zoom_start=6)
j = 1 
for index, row in ranking2.iterrows():
  icon=folium.features.CustomIcon('D:/Geospatial_Project/Medaglie/' + str(j) + '.png' ,icon_size=(40,40))
  marker = getMarker(row['geometry'].centroid.y,row['geometry'].centroid.x, row['TERRITORIO'], icon)
  marker.add_to(m7)
  j += 1